In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 4.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15819, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 15819 (delta 31), reused 47 (delta 17), pack-reused 15743 (from 1)
Receiving objects: 100% (15819/15819), 6.38 MiB | 11.71 MiB/s, done.
Resolving deltas: 100% (11525/11525), done.


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.0 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=2a23b5ab4b0b2c8a81e7be16c34aafce8133a3ceab267821c5f66918b377c5c7
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.5 ; cuda:  cu121
detectron2: 0.6


In [ ]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import glob
import os
import ntpath
import numpy as np
import cv2
import random
import itertools
import pandas as pd
from tqdm import tqdm
import urllib
import json
import PIL.Image as Image

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
!gdown --id 1K79wJgmPTWamqb04Op2GxW0SW9oxw8KS

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1K79wJgmPTWamqb04Op2GxW0SW9oxw8KS
To: /content/face_detection.json
100% 274k/274k [00:00<00:00, 6.46MB/s]


In [ ]:
faces_df = pd.read_json('face_detection.json', lines=True)

In [ ]:
faces_df.head()

,content,annotation,extras
0,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['Face'], 'notes': '', 'points': [{...",NaN
1,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['Face'], 'notes': '', 'points': [{...",NaN
2,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['Face'], 'notes': '', 'points': [{...",NaN
3,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['Face'], 'notes': '', 'points': [{...",NaN
4,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['Face'], 'notes': '', 'points': [{...",NaN


In [ ]:


import numpy as np
import requests
from tqdm import tqdm
from PIL import Image
from io import BytesIO



response = requests.get(faces_df[0]['content'])

TypeError: string indices must be integers

In [ ]:
import os
import pandas as pd
import urllib.request
from PIL import Image
from tqdm import tqdm

# Ensure the faces directory exists
os.makedirs("faces", exist_ok=True)

# Initialize an empty dataset
dataset = []

# Add headers for HTTP request
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

# Iterate over the DataFrame rows
for index, row in tqdm(faces_df.iterrows(), total=faces_df.shape[0]):
    try:
        # Download the image from the URL
        img_url = row["content"]
        req = urllib.request.Request(img_url, headers=headers)
        print(img_url)
        img_response = urllib.request.urlopen(req)

        # Open and process the image
        img = Image.open(img_response)
        img = img.convert('RGB')  # Convert to RGB format

        # Generate an image file name
        image_name = f'face_{index}.jpeg'

        # Save the image to the 'faces' directory
        img.save(f'faces/{image_name}', "JPEG")

        # Extract annotations
        annotations = row['annotation']
        for an in annotations:
            data = {}

            # Extract image dimensions and bounding box points
            width = an['imageWidth']
            height = an['imageHeight']
            points = an['points']

            # Prepare the dataset entry
            data['file_name'] = image_name
            data['width'] = width
            data['height'] = height
            data["x_min"] = int(round(points[0]["x"] * width))
            data["y_min"] = int(round(points[0]["y"] * height))
            data["x_max"] = int(round(points[1]["x"] * width))
            data["y_max"] = int(round(points[1]["y"] * height))
            data['class_name'] = 'face'

            # Append the data entry to the dataset
            dataset.append(data)
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Create a DataFrame from the dataset
df = pd.DataFrame(dataset)

# Save the DataFrame to a CSV file for future use
df.to_csv("faces_dataset.csv", index=False)

# Output some information about the dataset
print(f"Unique file names: {df.file_name.unique().shape[0]}, Total annotations: {df.shape[0]}")

  0%|          | 1/409 [00:00<01:11,  5.68it/s]

http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/8186c3d1-e9d4-4550-8ec1-a062a7628787___0-26.jpg.jpeg
Error processing row 0: HTTP Error 403: Forbidden
http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/d1c32c8e-8050-482d-a6c8-b101ccba5b65___0de0ee708a4a47039e441d488615ebb7.png


  1%|          | 3/409 [00:00<00:56,  7.18it/s]

Error processing row 1: HTTP Error 403: Forbidden
http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/aaa5a997-b248-4724-9002-b29117725102___0rCMbYS7vkSINlu61.jpg.jpeg
Error processing row 2: HTTP Error 403: Forbidden
http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/c4efa88f-8218-4675-b5ce-30a03bf8b8f4___1.png


  1%|          | 5/409 [00:00<00:53,  7.57it/s]

...

  5%|▍         | 19/409 [00:02<00:44,  8.72it/s]

Error processing row 17: HTTP Error 403: Forbidden
http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/584e1406-faf8-4613-9e9d-4e79620a2f44___6c9f5fafffdf652e5bbf672fd355e4de.jpg.jpeg
Error processing row 18: HTTP Error 403: Forbidden
http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/14d82e7b-17b5-48ed-b755-777921f8fe58___07d4fe77dce1c871944c80c3f5596b5c--funny-movie-quotes-funny-movies.jpg.jpeg


  5%|▍         | 20/409 [00:02<00:47,  8.23it/s]

Error processing row 19: HTTP Error 403: Forbidden
http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/5dcb5230-c3d2-4f78-8c9a-37c35e852289___7c9e84ab6a1d7cd45032c69f612893bc--the-train-cousin.jpg.jpeg


KeyboardInterrupt: 

In [ ]:
! wget 	http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/8186c3d1-e9d4-4550-8ec1-a062a7628787___0-26.jpg

--2024-12-13 14:10:24--  http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/8186c3d1-e9d4-4550-8ec1-a062a7628787___0-26.jpg
Resolving com.dataturks.a96-i23.open.s3.amazonaws.com (com.dataturks.a96-i23.open.s3.amazonaws.com)... 3.5.28.125, 52.216.108.147, 52.217.113.241, ...
Connecting to com.dataturks.a96-i23.open.s3.amazonaws.com (com.dataturks.a96-i23.open.s3.amazonaws.com)|3.5.28.125|:80... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-12-13 14:10:24 ERROR 403: Forbidden.



In [ ]:
wget --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36" "http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/8186c3d1-e9d4-4550-8ec1-a062a7628787___0-26.jpg" -O faces/0-26.jpg


SyntaxError: invalid imaginary literal (<ipython-input-24-fd6712758ad9>, line 1)

In [ ]:
!mkdir -p faces
!wget --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36" \
"http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/8186c3d1-e9d4-4550-8ec1-a062a7628787___0-26.jpg" -O faces/0-26.jpg


--2024-12-13 14:13:12--  http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/8186c3d1-e9d4-4550-8ec1-a062a7628787___0-26.jpg
Resolving com.dataturks.a96-i23.open.s3.amazonaws.com (com.dataturks.a96-i23.open.s3.amazonaws.com)... 52.217.165.129, 54.231.166.89, 16.15.193.37, ...
Connecting to com.dataturks.a96-i23.open.s3.amazonaws.com (com.dataturks.a96-i23.open.s3.amazonaws.com)|52.217.165.129|:80... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-12-13 14:13:12 ERROR 403: Forbidden.



In [ ]:
import os
import requests
from tqdm import tqdm

# Ensure the faces directory exists
os.makedirs("faces", exist_ok=True)

# Function to download images
def download_image(url, output_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers, stream=True)
        if response.status_code == 200:
            with open(output_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
        else:
            print(f"Failed to download {url}: HTTP {response.status_code}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

# Loop through the URLs and download
for index, url in tqdm(enumerate(faces_df['content'].tolist()), total=len(faces_df)):
    output_path = os.path.join("faces", f"face_{index}.jpg")
    download_image(url, output_path)


  0%|          | 2/409 [00:00<00:52,  7.71it/s]

Failed to download http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/8186c3d1-e9d4-4550-8ec1-a062a7628787___0-26.jpg.jpeg: HTTP 403
Failed to download http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/d1c32c8e-8050-482d-a6c8-b101ccba5b65___0de0ee708a4a47039e441d488615ebb7.png: HTTP 403


  1%|          | 4/409 [00:00<00:50,  8.02it/s]

Failed to download http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/aaa5a997-b248-4724-9002-b29117725102___0rCMbYS7vkSINlu61.jpg.jpeg: HTTP 403
Failed to download http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/c4efa88f-8218-4675-b5ce-30a03bf8b8f4___1.png: HTTP 403


  1%|▏         | 6/409 [00:00<00:49,  8.07it/s]

...

 16%|█▌        | 64/409 [00:07<00:39,  8.75it/s]

Failed to download http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/0b61c2de-be60-4436-b32a-c2b75f7b1376___23181bdffe3e99a98d87cf92cc2d2fff--movie-scenes-funny-funny-movies.jpg.jpeg: HTTP 403
Failed to download http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/6be2ed18-bf8f-4284-bf99-f4ac4fa7eb19___24861_full.jpg.jpeg: HTTP 403


 16%|█▌        | 65/409 [00:07<00:40,  8.41it/s]

Failed to download http://com.dataturks.a96-i23.open.s3.amazonaws.com/2c9fafb064277d86016431e33e4e003d/78dfed3c-0cae-4041-b86c-1364f7ee5f3c___57941e88dd0895057c8b4803-750-312.png: HTTP 403


KeyboardInterrupt: 